<a href="https://colab.research.google.com/github/jayjonas1996/twitter-hate-speech-detection/blob/main/ML_Proj_Hate_speech_detector_(CNN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Pinning the important libraries to a specific version
!pip install keras==1.2.2 --no-color
!pip install tensorflow==1.15.2 --no-color

     |████████████████████████████████| 175 kB 4.8 MB/s 
     |████████████████████████████████| 2.8 MB 35.5 MB/s 
  Created wheel for keras: filename=Keras-1.2.2-py3-none-any.whl size=209601 sha256=c54cbd8358bef8869d39d3d51742af8b187bcb166f0fa1c3de03fd677a895b76
  Stored in directory: /root/.cache/pip/wheels/d1/32/23/2a1db3765ec19c91503843380a4f92b6530598949c661c5fa2
  Created wheel for theano: filename=Theano-1.0.5-py3-none-any.whl size=2668111 sha256=49ae7a18eab9068ca0812f57b22c8fc2ec116a173ba6c00cc45773f431c09967
  Stored in directory: /root/.cache/pip/wheels/26/68/6f/745330367ce7822fe0cd863712858151f5723a0a5e322cc144
Successfully built keras theano
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.

Importing the necessary libraries

In [ ]:
import sys
import re
from string import punctuation
from collections import defaultdict

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import gensim
from gensim.parsing.preprocessing import STOPWORDS

import keras
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils
from keras.layers import Embedding, Input, LSTM
from keras.models import Sequential, Model
from keras.layers import Activation, Dense, Dropout, Embedding, \
                         Input, Convolution1D, MaxPooling1D, \
                         GlobalMaxPooling1D, Merge, InputLayer

import sklearn
from sklearn.model_selection import KFold
from sklearn.metrics import make_scorer, f1_score, accuracy_score, recall_score, \
                            precision_score, classification_report, \
                            precision_recall_fscore_support

from google.colab import drive
drive.mount("/content/gdrive")

Using TensorFlow backend.


Mounted at /content/gdrive


In [ ]:
EMBEDDING_DIM = 25
LOSS_FUN = "categorical_crossentropy"
OPTIMIZER = "adam"
NO_OF_FOLDS = 5
EPOCHS = 5
BATCH_SIZE = 128
FLAGS = re.MULTILINE | re.DOTALL

np.random.seed(42)

In [ ]:
# Import the dataset

GLOVE_MODEL_FILE = "/content/gdrive/My Drive/Dataset/glove.twitter.27B.25d.txt"
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format(GLOVE_MODEL_FILE)

df = pd.read_csv('/content/gdrive/My Drive/Dataset/twitter_hate_speech_data.csv')
df.head(5)

,label,tweet
0,0,@user when a father is dysfunctional and is s...
1,0,@user @user thanks for #lyft credit i can't us...
2,0,bihday your majesty
3,0,#model i love u take with u all the time in ...
4,0,factsguide: society now #motivation


In [ ]:
def hashtag(text):
    text = text.group()
    hashtag_body = text[1:]
    if hashtag_body.isupper():
        result = u"<hashtag> {} <allcaps>".format(hashtag_body)
    else:
        result = " ".join(["<hashtag>"] + re.split(r'(?=[A-Z])', hashtag_body))
    return result

def allcaps(text):
    text = text.group()
    return text.lower() + " <allcaps>"


def re_sub(text, pattern, repl):
    return re.sub(pattern, repl, text, flags=FLAGS)

def tokenizer_g(text):
    # Different regex parts for smiley faces
    eyes = r"[8:=;]"
    nose = r"['`\-]?"

    text = re_sub(text, r"https?:\/\/\S+\b|www\.(\w+\.)+\S*", "<url>")
    text = re_sub(text, r"/"," / ")
    text = re_sub(text, r"@\w+", "<user>")
    text = re_sub(text, r"{}{}[)dD]+|[)dD]+{}{}".format(eyes, nose, nose, eyes), "<smile>")
    text = re_sub(text, r"{}{}p+".format(eyes, nose), "<lolface>")
    text = re_sub(text, r"{}{}\(+|\)+{}{}".format(eyes, nose, nose, eyes), "<sadface>")
    text = re_sub(text, r"{}{}[\/|l*]".format(eyes, nose), "<neutralface>")
    text = re_sub(text, r"<3","<heart>")
    text = re_sub(text, r"[-+]?[.\d]*[\d]+[:,.\d]*", "<number>")
    text = re_sub(text, r"#\S+", hashtag)
    text = re_sub(text, r"([!?.]){2,}", r"\1 <repeat>")
    text = re_sub(text, r"\b(\S*?)(.)\2{2,}\b", r"\1\2 <elong>")

    text = re_sub(text, r"([A-Z]){2,}", allcaps)

    return text.lower()

def glove_tokenize(text):
    text = tokenizer_g(text)
    text = ''.join([c for c in text if c not in punctuation])
    words = text.split()
    words = [word for word in words if word not in STOPWORDS]
    return words


def gen_vocab(tweets_df):
    vocab, reverse_vocab = {}, {}
    freq = defaultdict(int)
    vocab_index = 1
    for index, row in tweets_df.iterrows():
        words = glove_tokenize(row['tweet'].lower())

        for word in words:
            if word not in vocab:
                vocab[word] = vocab_index
                reverse_vocab[vocab_index] = word       # generate reverse vocab as well
                vocab_index += 1
            freq[word] += 1
    vocab['UNK'] = len(vocab) + 1 # count unknown words
    reverse_vocab[len(vocab)] = 'UNK'
    return vocab, reverse_vocab, freq



In [ ]:
vocab, reverse_vocab = {}, {}
freq = defaultdict(int)
vocab_index = 1


for index, row in df.iterrows():
    _emb = 0
    words = glove_tokenize(row['tweet'].lower())

    for word in words:
        # Check if embeeding there in GLove model
        if word in word2vec_model:
            _emb += 1

        # Drop the rows where no word is found in glove after cleanup 
        # because of empty or useless small tweet 
        if not _emb and index in df.index:
          df.drop(index, inplace=True, axis=0)

        # Add words to new vocab
        if word not in vocab:
            vocab[word] = vocab_index
            reverse_vocab[vocab_index] = word
            vocab_index += 1
        freq[word] += 1
          
    # count unknown words
    vocab['UNK'] = len(vocab) + 1
    reverse_vocab[len(vocab)] = 'UNK'

def shuffle_weights(model):
    weights = model.get_weights()
    weights = [np.random.permutation(w.flat).reshape(w.shape) for w in weights]
    model.set_weights(weights)

def batch_gen(X, batch_size):
    import math
    n_batches = int(math.ceil(X.shape[0] / float(batch_size)))
    end = int(X.shape[0] / float(batch_size)) * batch_size
    n = 0
    for i in range(0, n_batches):
        if i < n_batches - 1: 
            batch = X[i*batch_size:(i+1) * batch_size, :]
            yield batch
        
        else:
            batch = X[end: , :]
            n += X[end:, :].shape[0]
            yield batch

In [ ]:
def cnn_model(sequence_length, embedding_dim):
    # Model Hyperparameters
    n_classes = 2
    filter_sizes = [1, 1, 1]
    num_filters = 100
    dropout_prob = [0.25, 0.5]

    cnn_layer_in = Input(shape=(sequence_length, embedding_dim))
    convs = []
    for filter_size in filter_sizes:
        conv = Convolution1D(nb_filter=num_filters,
                             filter_length=filter_size,
                             activation='relu')(cnn_layer_in)
        pool = GlobalMaxPooling1D()(conv)
        convs.append(pool)

    out = Merge(mode='concat')(convs)
    cnn_layer = Model(input=cnn_layer_in, output=out)

    # Main sequential model
    model = Sequential()
    
    model.add(Embedding(len(vocab) + 1, embedding_dim, input_length=sequence_length, trainable=None))
    model.add(Dropout(dropout_prob[0]))
    model.add(cnn_layer)
    model.add(Dropout(dropout_prob[1]))
    model.add(Activation('relu'))
    model.add(Dense(n_classes))
    model.add(Activation('softmax'))
    model.compile(loss=LOSS_FUN, optimizer=OPTIMIZER, metrics=['accuracy'])
    print(model.summary())
    return model

def lstm_model(sequence_length, embedding_dim):
    n_classes = 2
    model = Sequential()
    model.add(Embedding(len(vocab) + 1, embedding_dim, input_length=sequence_length, trainable=None))
    model.add(Dropout(0.25))
    model.add(LSTM(50))
    model.add(Dropout(0.5))
    model.add(Dense(n_classes, activation='softmax'))
    model.compile(loss=LOSS_FUN, optimizer=OPTIMIZER, metrics=['accuracy'])

    print(model.summary())
    return model

In [ ]:
def train_model(X, y, model, inp_dim, epochs=EPOCHS, batch_size=BATCH_SIZE):
    folds = KFold(n_splits=NO_OF_FOLDS, shuffle=True, random_state=42)
    print(folds)

    tweet_length = X.shape[1]
    total_loss, total_acc = [], []
    
    for train_index, test_index in folds.split(X):
        shuffle_weights(model)

        X_train, y_train = X[train_index], y[train_index]
        X_test, y_test = X[test_index], y[test_index]

        y_train = y_train.reshape((len(y_train), 1))
        X_y_stacked = np.hstack((X_train, y_train))
        for epoch in range(epochs):
            print(f'EPOCH {epoch + 1}')
            for batch in batch_gen(X_y_stacked, batch_size):
                X_batch, y_batch = batch[:, :tweet_length], batch[:, tweet_length]
                try:
                    y_batch = np_utils.to_categorical(y_batch, nb_classes=2)
                except Exception as e:
                    print(e)

                loss, acc = model.train_on_batch(X_batch, y_batch)
                total_loss.append(loss)
                total_acc.append(acc)

        y_pred = model.predict_on_batch(X_test)
        y_pred = np.argmax(y_pred, axis=1)
        print(classification_report(y_test, y_pred))
    
    return total_loss, total_acc

def plot_loss_acc(total_loss, total_acc):
  # summarize history for accuracy and loss
  plt.plot(total_loss)
  plt.title('Training loss')
  plt.ylabel('Loss')
  plt.xlabel('batch')
  plt.show()

  plt.plot(total_acc)
  plt.title('Training accuracy')
  plt.ylabel('Accuracy')
  plt.xlabel('batch')

  plt.show()


In [ ]:

X, y = [], []
for index, row in df.iterrows():
    text = glove_tokenize(row['tweet'].lower())
    text = ' '.join([c for c in text if c not in punctuation])
    words = text.split()
    words = [word for word in words if word not in STOPWORDS]
    seq, _emb = [], []
    for word in words:
        seq.append(vocab.get(word, vocab['UNK']))
    X.append(seq)
    y.append(row['label'])

print(max([len(x) for x in X]))
data = pad_sequences(X, maxlen=max([len(x) for x in X]))
print(data[88])
y = np.array(y)

41
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0 116  30 421 570  30 571 572 376  30 318 376
 573  30   8 569   1]


In [ ]:
data, y = sklearn.utils.shuffle(data, y)
cnn = cnn_model(data.shape[1], EMBEDDING_DIM)

# from keras.utils.visualize_util import plot
# plot(cnn, to_file='model_cnn.png')
# from google.colab import files
# files.download('model_cnn.png') 

total_loss, total_acc = train_model(data, y, cnn, EMBEDDING_DIM)
plot_loss_acc(total_loss, total_acc)

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_5 (Embedding)          (None, 41, 25)        1482275     embedding_input_5[0][0]          
____________________________________________________________________________________________________
dropout_9 (Dropout)              (None, 41, 25)        0           embedding_5[0][0]                
____________________________________________________________________________________________________
model_5 (Model)                  (None, 300)           7800        dropout_9[0][0]                  
____________________________________________________________________________________________________
dropout_10 (Dropout)             (None, 300)           0           model_5[1][0]                    
___________________________________________________________________________________________

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
data, y = sklearn.utils.shuffle(data, y)
lstm = lstm_model(data.shape[1], EMBEDDING_DIM)

# from keras.utils.visualize_util import plot
# plot(cnn, to_file='model_lstm.png')
# from google.colab import files
# files.download('model_lstm.png')

total_loss, total_acc = train_model(data, y, lstm, EMBEDDING_DIM)
plot_loss_acc(total_loss, total_acc)

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_6 (Embedding)          (None, 41, 25)        1482275     embedding_input_6[0][0]          
____________________________________________________________________________________________________
dropout_11 (Dropout)             (None, 41, 25)        0           embedding_6[0][0]                
____________________________________________________________________________________________________
lstm_1 (LSTM)                    (None, 50)            15200       dropout_11[0][0]                 
____________________________________________________________________________________________________
dropout_12 (Dropout)             (None, 50)            0           lstm_1[0][0]                     
___________________________________________________________________________________________

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>